this script is an attempt to classify good and bad translations from RusLTC on bilingual vectorised representations of lempos (no stopwords) in sources and targets

supposedly in this set up the algorithm captures accuracy of the translation rather than (morphosyntactic) fluency

Ideas to try out after the failure of the initial naive approach that also lacked in theoretical justification (concatenated ST and TT, represented with respective bilingual embeddings and padded to the max text length treated as instances)

1. sentence-level classification
2. two inputs to the LSTM

In [1]:
import io ### https://docs.python.org/3/library/io.html
import sys,os
import pandas as pd
import time
import random

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate
import gensim
from collections import OrderedDict

# TensorFlow and tf.keras
import logging

import tensorflow as tf
from tensorflow import keras

from keras.callbacks import TensorBoard, EarlyStopping
from keras import backend, preprocessing
from keras.layers import Dense, Input, LSTM, Bidirectional
from keras.models import Model
from keras.models import load_model as load_keras_model
from keras.layers import Embedding
from keras.utils.vis_utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import pydot
from keras.utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

print(tf.__version__)

## to avoid reloading kernel after changes to imported modules
# import importlib
# import HTQ_functions as mm
# importlib.reload(mm)

# import functions as mm1
# importlib.reload(mm1)

## import the functions from the helper scripts
# from acc_functions import 


%matplotlib inline
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

1.12.0


Using TensorFlow backend.


In [2]:
# def get_embeddings(sentences, model):
#     embeddings = model.encode(sentences, batch_size=128)
#     return embeddings

In [3]:
def preparedata(directory):
    ourdic = []
    print('Collecting data from the files...')
    for subdir in os.listdir(directory):
        files = [f for f in os.listdir(os.path.join(directory, subdir)) if f.endswith('.conllu')]
        for f in files:
            rowdic = {'doc': f.strip(), 'group': subdir}
            doc = open(os.path.join(directory, subdir, f))
            text = doc.read().strip() #.replace('\n', ' ')
            doc.close()
            rowdic['text'] = text
            ourdic.append(rowdic)
    ourdic = pd.DataFrame(ourdic)
    return ourdic

In [4]:
def visual(data, labels, classes):
    # Here goes the 2-D plotting of the data...
    pca = PCA(n_components=2)
    x_r = pca.fit_transform(data)
    plt.figure()
    # consistent colors
    cols = ['red', 'green', 'orange', 'blue', 'grey']
    colors = {}
    for i,name in enumerate(classes):
        colors[name] = cols[i]
#     colors = {'bad': 'red', 'good': 'green'}
    lw = 2

    for target_name in classes:
        plt.scatter(x_r[labels == target_name, 0], x_r[labels == target_name, 1], s=1, color=colors[target_name],
                    label=target_name, alpha=.8, lw=lw)
    plt.legend(loc='best', scatterpoints=1, prop={'size': 15})
    plt.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')
    plt.tick_params(axis='y', which='both', left='off', right='off', labelleft='off')
    #plt.savefig('plot.png', dpi=300)
    plt.show()
    plt.close()
    return x_r

In [5]:
## preprosessing from https://github.com/akutuzov/webvectors/blob/master/preprocessing/rus_preprocessing_udpipe.py
datadir = '/home/masha/accuracy/data/lempos/'  # The path to where subdirectories whith text files are
df = preparedata(datadir)

**=====transform the df to align each target with its source: doc, src, tgt, label=====**

In [6]:
targets_df = df[~(df['group']=='source')]
print(targets_df.shape)
sources_df = df[df['group']=='source']
sources_df = sources_df.sort_values('doc')
sources_df.head()

(542, 3)


,doc,group,text
608,EN_1_101.conllu,source,pussy_PROPN riot_PROPN putin_ADJ v_NOUN punk_N...
580,EN_1_102.conllu,source,anti_PROPN drug_PROPN agency_PROPN seeks_PROPN...
576,EN_1_114.conllu,source,product_NOUN placement_NOUN on_ADV broadway_VE...
557,EN_1_116.conllu,source,d_PROPN 'oh_PROPN german_PROPN broadcaster_PRO...
615,EN_1_121.conllu,source,how_ADV be_AUX you_PRON social_PROPN security_...


In [7]:
aligned = targets_df.copy()
aligned.insert(loc=2, column='source', value=None)
aligned.columns = ['doc', 'group', 'source', 'target']

aligned.head()

,doc,group,source,target
0,RU_1_244_35.conllu,bad,None,весь_DET правда_NOUN о_ADP гмый_PROPN сторонни...
1,RU_1_271_4.conllu,bad,None,не_PART посещать_VERB 'д_NOUN странный_ADJ реб...
2,RU_1_150_2.conllu,bad,None,последний_ADJ книга_NOUN роджер::пенроуз_PROPN...
3,RU_1_244_1.conllu,bad,None,весь_DET правда_NOUN о_ADP генетически_ADV мод...
4,RU_1_274_41.conllu,bad,None,газета_PROPN нью-йорк::таймз_PROPN xx_NUM октя...


In [8]:
aligned['temp'] = aligned['doc'].replace(to_replace='RU',value=r'EN', regex=True)
aligned['temp'] = aligned['temp'].replace(to_replace='_\d+\.conllu',value=r'.conllu', regex=True)
aligned = aligned.sort_values('temp')
aligned.head()

,doc,group,source,target,temp
254,RU_1_101_9.conllu,good,None,pussy_PROPN riot_PROPN путин_PROPN против_ADP ...,EN_1_101.conllu
62,RU_1_101_3.conllu,bad,None,пусси::райот_PROPN путин_PROPN против_ADP панк...,EN_1_101.conllu
11,RU_1_101_8.conllu,bad,None,пусси::райот_PROPN судебный_ADJ разбирательств...,EN_1_101.conllu
471,RU_1_102_6.conllu,good,None,служба_NOUN по_ADP борьба_NOUN с_ADP наркотик_...,EN_1_102.conllu
119,RU_1_102_15.conllu,bad,None,управление_PROPN по_ADP борьба_NOUN с_ADP нарк...,EN_1_102.conllu


In [9]:
sfns = aligned['temp'].tolist()
for i in sfns:
## two ways of putting the value from one column (text) in one df on the value in the other column (temp filename) into the specified column (source) of the other df, following the list respecting the order of the cells 
#     aligned.loc[aligned.temp == i, 'source'] = sources_df[sources_df['doc']=='EN_1_101.conllu']['text'].values[0]
    aligned.loc[aligned.temp == i, 'source'] = sources_df.loc[sources_df['doc'] == i, 'text'].item()
aligned.head()

,doc,group,source,target,temp
254,RU_1_101_9.conllu,good,pussy_PROPN riot_PROPN putin_ADJ v_NOUN punk_N...,pussy_PROPN riot_PROPN путин_PROPN против_ADP ...,EN_1_101.conllu
62,RU_1_101_3.conllu,bad,pussy_PROPN riot_PROPN putin_ADJ v_NOUN punk_N...,пусси::райот_PROPN путин_PROPN против_ADP панк...,EN_1_101.conllu
11,RU_1_101_8.conllu,bad,pussy_PROPN riot_PROPN putin_ADJ v_NOUN punk_N...,пусси::райот_PROPN судебный_ADJ разбирательств...,EN_1_101.conllu
471,RU_1_102_6.conllu,good,anti_PROPN drug_PROPN agency_PROPN seeks_PROPN...,служба_NOUN по_ADP борьба_NOUN с_ADP наркотик_...,EN_1_102.conllu
119,RU_1_102_15.conllu,bad,anti_PROPN drug_PROPN agency_PROPN seeks_PROPN...,управление_PROPN по_ADP борьба_NOUN с_ADP нарк...,EN_1_102.conllu


In [27]:
ru_xtrain0 = aligned['target']
en_xtrain0 = aligned['source']

yy_train = aligned['group'].tolist()

classes = sorted(list(set(aligned['group'].tolist())))
num_classes = len(classes)
logger.info('Number of classes: %d' % num_classes)
print('===========================')
print('Distribution of classes in the dataset:')
print(aligned.groupby('group').count())
print('===========================')

2019-10-09 17:48:26,091 : INFO : Number of classes: 2


Distribution of classes in the dataset:
       doc  source  target  temp
group                           
bad    213     213     213   213
good   329     329     329   329


In [28]:
## Have a look at a random document 
doc = random.choice(range(20))
# print(type(ru_xtrain0[doc]))
print('Random target doc:', ru_xtrain0[doc].split()[:20])
## or at the first text to avoid an extra variable
print('\nFirst target doc:', ru_xtrain0[0].split()[:20])

Random target doc: ['ученый_PROPN', 'против_ADP', 'нобелевский_ADJ', 'премия_NOUN', 'нобелевский_ADJ', 'премия_NOUN', 'по_ADP', 'психология_NOUN', 'медицина_NOUN', 'физик_NOUN', 'или_CCONJ', 'химия_NOUN', 'это_PRON', 'самый_ADJ', 'авторитетный_ADJ', 'и_CCONJ', 'почетный_ADJ', 'награда_NOUN', 'в_ADP', 'область_NOUN']

First target doc: ['весь_DET', 'правда_NOUN', 'о_ADP', 'гмый_PROPN', 'сторонник_NOUN', 'генномодифицировать_ADJ', 'зерно_NOUN', 'настаивать_VERB', 'что_SCONJ', 'это_PRON', 'единственный_ADJ', 'способ_NOUN', 'справляться_VERB', 'с_ADP', 'то_PRON', 'что_SCONJ', 'прокармливать_VERB', 'густонаселенный_ADJ', 'мир_NOUN', 'критика_NOUN']


**=====a neural model with an biLSTM layer=======** 

that is going to look at every word in ST+TT represented by corresponding bilingual embeddings of size 300

In [12]:
### concatenate embeddings externally and use the new embeddings file in this function (mind the necessity to update the first line and delete the first line of the appended file)
both = '/home/masha/MUSE/rig1_res/vectors0_en-ru.vec'
def load_embeddings(embeddings_file):
    # Определяем формат модели по её расширению:
    if embeddings_file.endswith('.bin.gz') or embeddings_file.endswith('.bin'):  # Бинарный формат word2vec
        emb_model = gensim.models.KeyedVectors.load_word2vec_format(
            embeddings_file, binary=True, unicode_errors='replace')
    elif embeddings_file.endswith('.txt.gz') or embeddings_file.endswith('.txt') \
            or embeddings_file.endswith('.vec.gz') or embeddings_file.endswith('.vec'):  # Текстовый формат word2vec
        emb_model = gensim.models.KeyedVectors.load_word2vec_format(
            embeddings_file, binary=False, unicode_errors='ignore') ##unicode_errors='replace'
    else:  # Нативный формат Gensim?
        emb_model = gensim.models.KeyedVectors.load(embeddings_file)
    emb_model.init_sims(replace=True)  # normalise vectors just in case
    return emb_model

bi_embeddings = load_embeddings(both)


logger.info('Loading word vectors')

## an alternative way to build model vocabulary
# ext_vocab = external_embeddings.index2entity
# ext_word_index = {}
#     for nr, word in enumerate(ext_vocab):
#         ext_word_index[word] = nr

voc = bi_embeddings.vocab
print(type(voc))
print(len(voc))
print(list(voc.keys())[0:5])
print(list(voc.keys())[400000:400005])

2019-10-09 17:40:19,768 : INFO : loading projection weights from /home/masha/MUSE/rig1_res/vectors0_en-ru.vec
2019-10-09 17:41:19,170 : WARNING : duplicate word 'клуб_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:19,185 : WARNING : duplicate word 'институт_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:19,202 : WARNING : duplicate word 'остров_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:19,203 : WARNING : duplicate word 'ii_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:19,205 : WARNING : duplicate word 'завод_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:19,212 : WARNING : duplicate word 'дивизия_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:19,233 : WARNING : duplicate word 'de_X' in /home/masha/

2019-10-09 17:41:21,114 : WARNING : duplicate word 'mac_PROPN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:21,119 : WARNING : duplicate word 'los_PROPN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:21,121 : WARNING : duplicate word '2.0_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:21,148 : WARNING : duplicate word '7,5_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:21,183 : WARNING : duplicate word '1,3_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:21,243 : WARNING : duplicate word 'от_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:21,270 : WARNING : duplicate word '0,2_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:21,278 : WARNING : duplicate word '-9_NUM' in /h

2019-10-09 17:41:22,462 : WARNING : duplicate word '2,1_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:22,481 : WARNING : duplicate word '3.0_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:22,490 : WARNING : duplicate word 'roman_PROPN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:22,509 : WARNING : duplicate word 'da_PROPN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:22,522 : WARNING : duplicate word 'pas_X' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:22,525 : WARNING : duplicate word 'per_X' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:22,548 : WARNING : duplicate word 'saint_PROPN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:22,553 : WARNING : duplicate word 'dos_PROPN' i

2019-10-09 17:41:23,591 : WARNING : duplicate word '4,6_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:23,596 : WARNING : duplicate word 'et_PROPN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:23,613 : WARNING : duplicate word 'es_X' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:23,618 : WARNING : duplicate word 'bei_X' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:23,626 : WARNING : duplicate word '5,6_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:23,632 : WARNING : duplicate word '1.0_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:23,657 : WARNING : duplicate word 'sp_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:23,673 : WARNING : duplicate word 'capitol_PROPN' in /ho

2019-10-09 17:41:24,988 : WARNING : duplicate word '7,8_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:25,026 : WARNING : duplicate word 'leo_PROPN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:25,057 : WARNING : duplicate word 'te_PROPN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:25,076 : WARNING : duplicate word '0,0_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:25,081 : WARNING : duplicate word 'a1_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:25,085 : WARNING : duplicate word 'ce_X' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:25,319 : WARNING : duplicate word 'ссср_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:41:25,327 : WARNING : duplicate word 're_PROPN' in /h

...

2019-10-09 17:42:11,308 : WARNING : duplicate word 'np_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:11,317 : WARNING : duplicate word '21.2_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:11,331 : WARNING : duplicate word 'flor_X' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:11,355 : WARNING : duplicate word 'wu_X' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:11,392 : WARNING : duplicate word '31_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:11,516 : WARNING : duplicate word 'iphone_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:11,520 : WARNING : duplicate word '6-9_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:11,603 : WARNING : duplicate word '31.1_NUM' in /ho

2019-10-09 17:42:13,201 : WARNING : duplicate word 'k8_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:13,207 : WARNING : duplicate word '3.25_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:13,239 : WARNING : duplicate word '1.88_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:13,248 : WARNING : duplicate word '3.14_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:13,254 : WARNING : duplicate word 'ers_PROPN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:13,272 : WARNING : duplicate word 'b4_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:13,276 : WARNING : duplicate word 'ddr3_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:13,294 : WARNING : duplicate word '2.22_NUM' 

2019-10-09 17:42:15,106 : WARNING : duplicate word 'mur_X' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:15,120 : WARNING : duplicate word 'palabras_X' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:15,136 : WARNING : duplicate word ''03_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:15,139 : WARNING : duplicate word 'null_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:15,184 : WARNING : duplicate word 'ex_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:15,223 : WARNING : duplicate word 'trad_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:15,227 : WARNING : duplicate word 'r4_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:15,274 : WARNING : duplicate word 'm41_NOUN' in

2019-10-09 17:42:16,726 : WARNING : duplicate word '700,000_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:16,750 : WARNING : duplicate word '28.1_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:16,767 : WARNING : duplicate word 'wta_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:16,791 : WARNING : duplicate word 'e85_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:16,805 : WARNING : duplicate word '802.3_NUM' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:16,826 : WARNING : duplicate word 'acp_NOUN' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:16,828 : WARNING : duplicate word 'palais_X' in /home/masha/MUSE/rig1_res/vectors0_en-ru.vec, ignoring all but first
2019-10-09 17:42:16,887 : WARNING : duplicate word 'nunca_

<class 'dict'>
542754
['Forfar_PROPN', 'Eadulf_PROPN', 'codice51_NOUN', 'Malm_PROPN', '106th_ADJ']
['17,9_NUM', 'Purnananda::Burhagohain_PROPN', 'разврат_NOUN', 'Varghese_PROPN', 'Firebird_PROPN']


In [13]:
import re

mvoc_words = list(voc.keys())
counter = 0
for w in mvoc_words:
    if re.findall(r"[а-я]_", w):
#         print(w)
        counter += 1
print(counter)
## do I have duplicate keys in this list? Which is to be expected because Russian embeddings might have English words in Latin script
print(len(mvoc_words) == len(set(mvoc_words)))
print(len(mvoc_words))
print('It looks like I dont have duplicates')

206685
True
542754
It looks like I dont have duplicates


In [29]:
# Convert class labels into indices
y_train0 = [classes.index(i) for i in yy_train]
print(y_train0[:5])

[1, 0, 0, 1, 0]


In [30]:
## Convert indices to categorical values to use with (categorical_)crossentropy loss
y_train = to_categorical(y_train0, num_classes)
print('We have a binary classification (see number of columns):', y_train.shape)

We have a binary classification (see number of columns): (542, 2)


In [16]:
def get_index(word, vocabulory=None):
    ## get word index in the models vocabulary
    if word in vocabulory:
        return vocabulory[word].index
    ## set index to OOV items like putin_ADJ to 0
    else:
        return 0

In [17]:
print('First source text:', aligned['source'][0].split()[:20])
print('\nThe first of the aligned translations:', aligned['target'][0].split()[:20])

First source text: ['the_DET', 'truth_NOUN', 'about_ADP', 'genetically_ADV', 'modify_ADJ', 'food_NOUN', 'proponent_NOUN', 'of_ADP', 'genetically_ADV', 'modify_VERB', 'crops_NOUN', 'say_VERB', 'the_DET', 'technology_NOUN', 'be_AUX', 'the_DET', 'only_ADJ', 'way_NOUN', 'to_PART', 'feed_VERB']

The first of the aligned translations: ['весь_DET', 'правда_NOUN', 'о_ADP', 'гмый_PROPN', 'сторонник_NOUN', 'генномодифицировать_ADJ', 'зерно_NOUN', 'настаивать_VERB', 'что_SCONJ', 'это_PRON', 'единственный_ADJ', 'способ_NOUN', 'справляться_VERB', 'с_ADP', 'то_PRON', 'что_SCONJ', 'прокармливать_VERB', 'густонаселенный_ADJ', 'мир_NOUN', 'критика_NOUN']


In [18]:
##convert them to embedding indices in a joint biling embedding file
entrain0 = [[get_index(w,voc) for w in text.split()] for text in aligned['source']]
rutrain0 = [[get_index(w,voc) for w in text.split()] for text in aligned['target']]

## here we are getting rid of OOV items:
entrain = [[id for id in text if id != 0] for text in entrain0]
rutrain = [[id for id in text if id != 0] for text in rutrain0]

print('First source text:', entrain0[0][:20])
print('First source text with no OOV (tot num of enOOV=123146! ruOOV=71558):', entrain[0][:20])

First source text: [334596, 322735, 0, 0, 3814, 0, 87, 0, 948, 0, 0, 0, 0, 163, 0, 5402, 9, 0, 807, 69]
First source text with no OOV (tot num of enOOV=123146! ruOOV=71558): [334596, 322735, 3814, 87, 948, 163, 5402, 9, 807, 69, 1215, 1545, 12455, 329916, 297289, 96613, 486159, 82718, 332744, 153]


In [19]:
logger.info('Average SOURCE text length: %s tokens'
                % "{0:.1f}".format(np.mean(list(map(len, entrain))), 1)) #aligned['source'].str.split()
logger.info('Average TARGET text length: %s tokens'
                % "{0:.1f}".format(np.mean(list(map(len, rutrain))), 1))
logger.info('Maximum SOURCE text length: %s tokens'
                % "{0:.1f}".format(np.max(list(map(len, entrain))), 1))
logger.info('Maximum TARGET text length: %s tokens'
                % "{0:.1f}".format(np.max(list(map(len, rutrain))), 1))

2019-10-09 17:46:29,977 : INFO : Average SOURCE text length: 226.1 tokens
2019-10-09 17:46:29,980 : INFO : Average TARGET text length: 257.7 tokens
2019-10-09 17:46:29,983 : INFO : Maximum SOURCE text length: 430.0 tokens
2019-10-09 17:46:29,984 : INFO : Maximum TARGET text length: 685.0 tokens


In [20]:
# Padding: make all texts lengths equal to absolute max in the data by filling in zeros
max_seq_length = 685  
vectorized_en = preprocessing.sequence.pad_sequences(
    entrain, maxlen=max_seq_length, truncating='post', padding='post')
vectorized_ru = preprocessing.sequence.pad_sequences(
    rutrain, maxlen=max_seq_length, truncating='post', padding='post')
print('The translation turned into %s ...,\n its length is %s' % (vectorized_ru[0][:5], len(vectorized_ru[0]))) #, 
print('Data parameters:', vectorized_ru.shape)

The translation turned into [334596 322735 302552 315239 298151] ...,
 its length is 685
Data parameters: (542, 685)


In [21]:
print(type(vectorized_en))
print(vectorized_en[:5])
print(len(vectorized_en))
print(len(vectorized_ru))
paired_vectorized = list(zip(list(vectorized_en), list(vectorized_ru)))

bitext_train = np.array([np.concatenate((p[0],p[1]), axis=None) for p in paired_vectorized])

print(len(bitext_train))
print(type(bitext_train))

<class 'numpy.ndarray'>
[[334596 322735   3814 ...      0      0      0]
 [334596 322735   3814 ...      0      0      0]
 [334596 322735   3814 ...      0      0      0]
 [359684 341992 328252 ...      0      0      0]
 [359684 341992 328252 ...      0      0      0]]
542
542
542
<class 'numpy.ndarray'>


**====BUILDING the model====**

In [22]:
def createmodel(loss='binary_crossentropy'): # 'binary_crossentropy', 'categorical_crossentropy'
    word_input = Input(shape=(max_seq_length*2,), name='Word_sequences')

    ## represent words in texts with their embeddings (what happens to OOVwords with index=0)

    vectors = bi_embeddings.vectors
    print(vectors.shape)
    ## Input shape: 2D tensor with shape: (batch_size, sequence_length)
    ## Output shape: 3D tensor with shape: (batch_size, sequence_length, output_dim)
    e = Embedding(input_dim=vectors.shape[0], output_dim=vectors.shape[1], weights=[vectors],
                    input_length=max_seq_length*2, trainable=False, name='embeddings')(word_input)
    # units is a hyperparameter; the output of the lstm layer
    x = Bidirectional(LSTM(units=128, name='biLSTM'))(e)  # this is where the model would process the input sequence in both directions

    output = Dense(num_classes, activation='softmax', name='Output')(x)  # this is the output layer which predicts the labels

    ## compile the model
    model = Model(inputs=[word_input], outputs=output)
    model.compile(loss=loss, optimizer='adam', metrics=['accuracy'])

    # print the model architecture
    print(model.summary())

    return model

In [23]:
model0 = createmodel()

(542754, 300)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Word_sequences (InputLayer)  (None, 1370)              0         
_________________________________________________________________
embeddings (Embedding)       (None, 1370, 300)         162826200 
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               439296    
_________________________________________________________________
Output (Dense)               (None, 2)                 514       
Total params: 163,266,010
Trainable params: 439,810
Non-trainable params: 162,826,200
_________________________________________________________________
None


In [32]:
# Train the model on our data
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

## compute the class weights to pass to the scoring function which generates the update weights (values)
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train0),
                                                 y_train0)
## convert class_weights to dict
class_weight_dict = dict(enumerate(class_weights))
print(class_weight_dict)

earlystopping = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=3, verbose=1, mode='max')
val_split = 0.1 

start = time.time()
## batch_size: number of samples per gradient update.
## verbosity: 0 = silent, 1 = progress bar, 2 = one line per epoch.
## validation_split: evaluate the loss and any model metrics on this data at the end of each epoch
## shuffle: whether to shuffle the training data before each epoch
## class_weight: "pay more attention" to samples from an under-represented class; https://datascience.stackexchange.com/questions/13490/how-to-set-class-weights-for-imbalanced-classes-in-keras
## {0: 2., 1: 1.} "treat every instance of class 0 (bad) as 2 instances of class 0" > the loss becomes a weighted average

X_train, X_test, Y_train, Y_test = train_test_split(bitext_train, y_train,
                                                    stratify=y_train, 
                                                    test_size=0.1, random_state=42)

history = model0.fit(X_train, Y_train, epochs=10, verbose=1, validation_data=(X_test,Y_test),
         batch_size=4, shuffle=True, class_weight=class_weight_dict, callbacks=[earlystopping]) ## class_weight=class_weight_dict, validation_split=val_split
end = time.time()
training_time = int(end - start)
logger.info('Trained in %s minutes' % str(round(training_time/60)))

{0: 1.272300469483568, 1: 0.8237082066869301}
Train on 487 samples, validate on 55 samples
Epoch 1/10
487/487 [==============================] - 122s 250ms/step - loss: 0.6924 - acc: 0.5257 - val_loss: 0.6610 - val_acc: 0.6364
Epoch 2/10
487/487 [==============================] - 113s 231ms/step - loss: 0.6439 - acc: 0.6160 - val_loss: 0.5954 - val_acc: 0.6545
Epoch 3/10
487/487 [==============================] - 115s 237ms/step - loss: 0.6018 - acc: 0.6283 - val_loss: 0.6114 - val_acc: 0.6727
Epoch 4/10
487/487 [==============================] - 107s 219ms/step - loss: 0.5763 - acc: 0.6448 - val_loss: 0.6006 - val_acc: 0.6545
Epoch 5/10
487/487 [==============================] - 109s 224ms/step - loss: 0.5567 - acc: 0.6632 - val_loss: 0.5848 - val_acc: 0.6545
Epoch 6/10
487/487 [==============================] - 107s 219ms/step - loss: 0.5454 - acc: 0.6879 - val_loss: 0.6404 - val_acc: 0.6364


2019-10-09 18:00:19,259 : INFO : Trained in 11 minutes


Epoch 00006: early stopping


In [33]:
# Evaluate the model:
print(model0.metrics_names)
score = model0.evaluate(X_test,Y_test, verbose=1)
logger.info('Loss function value: %s' % "{0:.4f}".format(score[0]))
logger.info('Accuracy on the test set: %s' % "{0:.4f}".format(score[1]))

['loss', 'acc']
55/55 [==============================] - 1s 20ms/step


2019-10-09 18:04:54,179 : INFO : Loss function value: 0.6404
2019-10-09 18:04:54,181 : INFO : Accuracy on the test set: 0.6364


In [34]:
# Используем функцию из sklearn чтобы посчитать F1 по каждому классу:
predictions = model0.predict(X_test)
predictions = np.around(predictions)  # проецируем предсказания модели в бинарный диапазон {0, 1}

In [35]:
# Конвертируем предсказания обратно из чисел в текстовые метки классов
y_test_real = [classes[np.argmax(pred)] for pred in Y_test]
predictions = [classes[np.argmax(pred)] for pred in predictions]

logger.info('Classification results on the test set:')
print(classification_report(y_test_real, predictions))
print("Confusion matrix\n", confusion_matrix(y_test_real, predictions))

fscore = precision_recall_fscore_support(y_test_real, predictions, average='macro')[2]
logger.info('Macro-F1 on the test set: %s' % "{0:.4f}".format(fscore))

2019-10-09 18:05:07,110 : INFO : Classification results on the test set:
2019-10-09 18:05:07,120 : INFO : Macro-F1 on the test set: 0.6333


              precision    recall  f1-score   support

         bad       0.54      0.68      0.60        22
        good       0.74      0.61      0.67        33

   micro avg       0.64      0.64      0.64        55
   macro avg       0.64      0.64      0.63        55
weighted avg       0.66      0.64      0.64        55

Confusion matrix
 [[15  7]
 [13 20]]
